In [113]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB


In [114]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df['flower_name'] = df.target.apply(lambda x: iris.target_names[x])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,flower_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica


In [115]:
clf= GridSearchCV(SVC(gamma='auto'),
                  {'C':[1,5,10,20,50],
                   'kernel':['rbf','linear']},
                   cv=5,return_train_score=False)

X = df.drop(['target', 'flower_name'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 5, 10, 20, 50], 'kernel': ['rbf', 'linear']})

In [116]:
dfr= pd.DataFrame(clf.cv_results_)
dfr[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.991667
1,1,linear,0.991667
2,5,rbf,0.983333
3,5,linear,0.991667
4,10,rbf,0.975000
5,10,linear,0.991667
6,20,rbf,0.975000
7,20,linear,0.983333
8,50,rbf,0.958333
9,50,linear,0.975000


In [117]:
model_params = {
    'svm': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10, 20],
            'kernel': ['linear', 'rbf', 'poly'],
            'gamma': ['auto']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [10, 50, 100, 150],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            'C': [0.1, 1, 10, 50]
        }
    
    },
    'k_neighbors': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 7, 15],
            'weights': ['uniform', 'distance']
        }
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 20, 50],
            'min_samples_split': [2, 10, 30]
        }
    },
    'naive_bayes': {
        'model': GaussianNB(),
        'params': {}
    }
    }

In [118]:
scores = []

for model_name, mp in model_params.items():
    clfr =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clfr.fit(X_train, y_train)    
    scores.append({
        'model': model_name,
        'best_score': clfr.best_score_,
        'best_params': clfr.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.991667,"{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}"
1,random_forest,0.966667,"{'max_depth': None, 'min_samples_split': 2, 'n..."
2,logistic_regression,0.966667,{'C': 10}
3,k_neighbors,0.983333,"{'n_neighbors': 7, 'weights': 'distance'}"
4,decision_tree,0.958333,"{'max_depth': None, 'min_samples_split': 2}"
5,naive_bayes,0.966667,{}
